In [ ]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# --- Configuration ---
# You can adjust these values
PIXELS_PER_FOOT = 15.0  # Approximate conversion factor, tune this for your video
PLAYER_CONFIDENCE_THRESHOLD = 0.5  # Min confidence to consider a person a potential player
MAIN_PLAYER_SELECTION_DELAY = 60  # Frames to wait before selecting main players
MAX_TRACKER_AGE = 30  # How many frames a track can be lost before it's deleted

# --- Initialization ---
# Load YOLO model
model = YOLO('yolov8n.pt')

# Initialize tracker
tracker = DeepSort(max_age=MAX_TRACKER_AGE)

# Tracking data
total_distance = {}
prev_positions = {}
main_players_ids = set()
frame_count = 0

# --- Video Handling ---
video_path = 'Tennis.mp4'
if not os.path.exists(video_path):
    print(f"Error: Video file not found at {video_path}")
    exit()

cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# --- Main Functions ---
def select_main_players(tracks):
    """
    Selects the 2 players closest to the bottom of the frame after a delay.
    This is more stable than using movement, which can be erratic at the start.
    """
    global main_players_ids
    
    # If players are already selected, keep tracking them
    if main_players_ids:
        current_ids = {t.track_id for t in tracks if t.is_confirmed()}
        # If we still see both players, no need to re-select
        if main_players_ids.issubset(current_ids):
            return [t for t in tracks if t.track_id in main_players_ids]

    # Only select after a certain number of frames for stability
    if frame_count > MAIN_PLAYER_SELECTION_DELAY:
        confirmed_tracks = [t for t in tracks if t.is_confirmed()]
        
        if len(confirmed_tracks) >= 2:
            # Sort tracks by the y-coordinate of their bounding box bottom (lower on screen)
            confirmed_tracks.sort(key=lambda t: t.to_ltwh()[1] + t.to_ltwh()[3], reverse=True)
            
            # Select the two lowest players
            top_2_players = confirmed_tracks[:2]
            main_players_ids = {t.track_id for t in top_2_players}
            print(f"Selected main players by position: {list(main_players_ids)}")
    
    return [t for t in tracks if t.track_id in main_players_ids]


# --- Main Loop ---
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    
    # 1. Detection
    results = model(frame, verbose=False)
    detections = []
    for box in results[0].boxes:
        # Filter for 'person' class with a certain confidence
        if int(box.cls) == 0 and float(box.conf) > PLAYER_CONFIDENCE_THRESHOLD:
            x1, y1, x2, y2 = box.xyxy[0]
            w, h = x2 - x1, y2 - y1
            detections.append(([float(x1), float(y1), float(w), float(h)],
                               float(box.conf),
                               'person'))

    # 2. Tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    # 3. Process Tracks and Calculate Distance
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        # Use the bottom-center of the bounding box as the player's position
        l, top, w, h = track.to_ltwh()
        cx, cy = l + w / 2, top + h

        # Calculate distance moved since last frame
        if track_id in prev_positions:
            px, py = prev_positions[track_id]
            # Euclidean distance in pixels
            dist_pixels = np.sqrt((cx - px)**2 + (cy - py)**2)
            # Convert to feet
            dist_feet = dist_pixels / PIXELS_PER_FOOT
            total_distance[track_id] = total_distance.get(track_id, 0) + dist_feet

        prev_positions[track_id] = (cx, cy)

    # 4. Identify Main Players
    main_player_tracks = select_main_players(tracks)
    # Sort by y-position to keep player 1 and 2 consistent
    main_player_tracks.sort(key=lambda t: t.to_ltwh()[1])

    # 5. Visualization
    player_names = ["Sharapova", "C. Wozniacki"]
    colors = [(255, 0, 255), (255, 255, 0)]  # Magenta and Cyan for visibility

    for i, track in enumerate(main_player_tracks):
        l, top, w, h = track.to_ltwh()
        track_id = track.track_id
        color = colors[i]
        
        # Player Bounding Box
        cv2.rectangle(frame, (int(l), int(top)), (int(l + w), int(top + h)), color, 2)
        
        # Player Info Text
        label_pos = (int(l), int(top) - 10)
        player_name = player_names[i]
        distance_text = f"Distance: {total_distance.get(track_id, 0):.2f} feet"
        
        cv2.putText(frame, player_name, label_pos, 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.putText(frame, distance_text, (label_pos[0], label_pos[1] - 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    # Display Frame
    cv2.imshow('Tennis Player Tracking', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# --- Cleanup ---
cap.release()
cv2.destroyAllWindows()

print("\n--- Final Distances for Main Players ---")
if main_players_ids:
    for i, track_id in enumerate(main_players_ids):
        player_name = player_names[i] if i < len(player_names) else f"Player {i+1}"
        distance = total_distance.get(track_id, 0)
        print(f"{player_name} (ID: {track_id}): {distance:.2f} feet")
else:
    print("No main players were selected.")

c:\Users\Aman Kumar\AppData\Local\Programs\Python\Python312\Lib\site-packages\deep_sort_realtime\embedder\embedder_pytorch.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Selected main players by position: ['8', '1']
Selected main players by position: ['14', '1']
Selected main players by position: ['3', '1']
Selected main players by position: ['44', '36']
Selected main players by position: ['44', '73']
Selected main players by position: ['44', '97']
Selected main players by position: ['97', '98']
Selected main players by position: ['102', '98']
Selected main players by position: ['102', '101']
Selected main players by position: ['101', '120']
Selected main players by position: ['140', '139']
Selected main players by position: ['139', '141']
Selected main players by position: ['144', '139']
Selected main players by position: ['157', '155']
Selected main players by position: ['153', '155']
Selected main players by position: ['159', '161']
Selected main players by position: ['161', '165']
Selected main players by position: ['166', '165']
Selected main players by position: ['186', '179']
Selected main players by position: ['190', '186']
Selected main player